In [18]:
# Step 1: Install libraries
!pip install faker pandas plotly

# Step 2: Import libraries
from faker import Faker
import pandas as pd
import random
import json
from datetime import datetime, timedelta
import sqlite3
import plotly.express as px
import plotly.graph_objects as go

# Initialize Faker
fake = Faker()

# GCC countries and some cities for locations
gcc_locations = [
    'Manama, Bahrain', 'Riffa, Bahrain', 'Muharraq, Bahrain',
    'Riyadh, Saudi Arabia', 'Jeddah, Saudi Arabia', 'Dammam, Saudi Arabia',
    'Dubai, UAE', 'Abu Dhabi, UAE', 'Sharjah, UAE',
    'Doha, Qatar', 'Al Rayyan, Qatar',
    'Kuwait City, Kuwait', 'Hawalli, Kuwait',
    'Muscat, Oman', 'Salalah, Oman'
]

# Step 3: Create common pool of 1000 account_numbers
account_numbers = [f'BH{random.randint(100000, 999999)}' for _ in range(1000)]

# Step 4: Generate CBS Data (1000 rows, JSON)
cbs_data = [{
    'transaction_id': f'TXN{i:05d}',
    'account_number': account_numbers[i % 1000],
    'amount': None if random.random() < 0.01 else round(random.uniform(10, 5000), 2),  # 1% missing
    'txn_type': random.choice(['Deposit', 'Withdrawal', 'Transfer']),
    'timestamp': (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d %H:%M:%S'),
    'merchant': None if random.random() < 0.02 else fake.company()  # 2% missing
} for i in range(1000)]
with open('cbs_data.json', 'w') as f:
    json.dump(cbs_data, f, indent=4)

# Step 5: Generate ATM/POS Data (1000 rows, CSV) with GCC locations
atm_data = [{
    'transaction_id': f'ATM{i:05d}',
    'account_number': account_numbers[i % 1000],
    'amount': None if random.random() < 0.01 else round(random.uniform(10, 1000), 2),
    'location': 'Not Provided' if random.random() < 0.05 else random.choice(gcc_locations),  # 5% 'Not Provided'
    'timestamp': (datetime.now() - timedelta(days=random.randint(0, 30))).strftime('%Y-%m-%d %H:%M:%S')
} for i in range(1000)]
df_atm = pd.DataFrame(atm_data)
df_atm.to_csv('atm_data.csv', index=False)

# Step 6: Generate CRM Data (1000 rows, CSV)
crm_data = [{
    'customer_id': f'CUST{i:03d}',
    'name': fake.name(),
    'address': fake.address().replace('\n', ', '),
    'account_number': account_numbers[i]
} for i in range(1000)]
df_crm = pd.DataFrame(crm_data)
df_crm.to_csv('crm_data.csv', index=False)

# Step 7: Ingestion - Load Data
with open('cbs_data.json', 'r') as f:
    cbs_data = json.load(f)
df_cbs = pd.DataFrame(cbs_data)
df_atm = pd.read_csv('atm_data.csv')
df_crm = pd.read_csv('crm_data.csv')

# Step 8: Merge Data
df_merged = df_cbs.merge(df_atm, on=['account_number', 'timestamp'], how='left', suffixes=('_cbs', '_atm'))
df_merged = df_merged.merge(df_crm, on='account_number', how='left')

# Step 9: Missing Values Visualization (Before Handling)
missing_before = df_merged.isna().sum().reset_index()
missing_before.columns = ['column', 'missing_count']
fig_missing_before = px.bar(
    missing_before,
    x='column',
    y='missing_count',
    title='Missing Values Before Handling',
    labels={'column': 'Column', 'missing_count': 'Missing Count'},
    color='column',
    text='missing_count',
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig_missing_before.update_layout(showlegend=False)
fig_missing_before.show()

# Step 10: Handle Missing Values
print("Missing Values Before:\n", df_merged.isna().sum())
df_merged = df_merged.dropna(subset=['account_number', 'amount_cbs'])
df_merged['merchant'] = df_merged['merchant'].fillna('Unknown')
df_merged['location'] = df_merged['location'].fillna('Not Provided')
df_merged['amount_atm'] = df_merged['amount_atm'].fillna(0)
print("Missing Values After:\n", df_merged.isna().sum())

# Step 11: Missing Values Visualization (After Handling)
missing_after = df_merged.isna().sum().reset_index()
missing_after.columns = ['column', 'missing_count']
fig_missing_after = px.bar(
    missing_after,
    x='column',
    y='missing_count',
    title='Missing Values After Handling',
    labels={'column': 'Column', 'missing_count': 'Missing Count'},
    color='column',
    text='missing_count',
    color_discrete_sequence=px.colors.qualitative.Plotly
)
fig_missing_after.update_layout(showlegend=False)
fig_missing_after.show()

# Step 12: Flag Potential Fraud (Strict Rules)
df_merged['fraud_risk'] = (
    (df_merged['merchant'] == 'Unknown') &
    (df_merged['amount_cbs'] > 4000) &
    (df_merged['location'] == 'Not Provided')
).astype(int)

# Step 13: Add Amount Category
df_merged['amount_category'] = pd.cut(df_merged['amount_cbs'],
                                     bins=[0, 500, 2000, float('inf')],
                                     labels=['Small', 'Medium', 'Large'])

# Step 14: Save to SQLite (Data Lake Simulation)
conn = sqlite3.connect('bank_data.db')
df_merged.to_sql('bank_transactions', conn, if_exists='replace', index=False)
conn.close()

# Step 15: Plotly Visualizations
# Viz 1: Transaction Count by Type (Bar)
txn_counts = df_merged['txn_type'].value_counts().reset_index()
txn_counts.columns = ['txn_type', 'count']
fig1 = px.bar(txn_counts, x='txn_type', y='count', title='Transaction Count by Type',
              labels={'txn_type': 'Transaction Type', 'count': 'Count'},
              color='txn_type', text='count')
fig1.update_layout(showlegend=False)
fig1.show()

# Viz 2: Amount Distribution (Histogram)
fig2 = px.histogram(df_merged, x='amount_cbs', nbins=30, title='Distribution of Transaction Amounts',
                    labels={'amount_cbs': 'Amount (BHD)'}, color_discrete_sequence=['blue'])
fig2.update_layout(bargap=0.1)
fig2.show()

# Viz 3: Top 5 Locations by Transaction Count (Polar Area)
top_locations = df_merged['location'].value_counts().reset_index()
top_locations.columns = ['location', 'count']
fig3 = px.bar_polar(
    top_locations,
    r='count',
    theta='location',
    title='Top 5 Locations by Transaction Count (Polar Area)',
    color='location',
    color_discrete_sequence=px.colors.qualitative.Plotly,
    template='plotly_dark'
)
fig3.update_traces(text=top_locations['count'])
fig3.update_layout(showlegend=True, polar=dict(radialaxis=dict(visible=True)))
fig3.show()

# Viz 4: Fraud Transactions by Location (Bar)
fraud_by_location = df_merged[df_merged['fraud_risk'] == 1]['location'].value_counts().head(5).reset_index()
fraud_by_location.columns = ['location', 'count']
fig4 = px.bar(fraud_by_location, x='location', y='count', title='Fraud Transactions by Location',
              labels={'location': 'Location', 'count': 'Number of Fraud Transactions'},
              color='location', text='count', color_discrete_sequence=['red'])
fig4.update_layout(showlegend=False)
fig4.show()

# Viz 5: Fraud vs Non-Fraud (Pie)
fraud_counts = df_merged['fraud_risk'].value_counts().reset_index()
fraud_counts.columns = ['fraud_risk', 'count']
fraud_counts['fraud_risk'] = fraud_counts['fraud_risk'].map({1: 'Risky', 0: 'Safe'})
fig5 = px.pie(fraud_counts, names='fraud_risk', values='count', title='Fraud Risk Distribution',
              color_discrete_sequence=['green', 'red'])
fig5.update_traces(textinfo='percent+label')
fig5.show()

# Step 16: Save Final Data with Insights
df_merged.to_csv('final_bank_data_with_insights.csv', index=False)

# Step 17: Summary
print(f"Total Transactions: {len(df_merged)}")
print(f"Total Amount: {df_merged['amount_cbs'].sum():.2f} BHD")
print(f"Fraud Transactions: {df_merged['fraud_risk'].sum()}")
print("Amount Category Distribution:\n", df_merged['amount_category'].value_counts())
print("Location Distribution Sample:\n", df_merged['location'].value_counts().head(10))

Missing Values Before:
 transaction_id_cbs      0
account_number          0
amount_cbs             14
txn_type                0
timestamp               0
merchant               14
transaction_id_atm    974
amount_atm            974
location              974
customer_id             0
name                    0
address                 0
dtype: int64
Missing Values After:
 transaction_id_cbs      0
account_number          0
amount_cbs              0
txn_type                0
timestamp               0
merchant                0
transaction_id_atm    961
amount_atm              0
location                0
customer_id             0
name                    0
address                 0
dtype: int64


Total Transactions: 990
Total Amount: 2475354.83 BHD
Fraud Transactions: 2
Amount Category Distribution:
 amount_category
Large     603
Medium    286
Small     101
Name: count, dtype: int64
Location Distribution Sample:
 location
Not Provided            962
Dammam, Saudi Arabia      6
Muharraq, Bahrain         3
Doha, Qatar               3
Abu Dhabi, UAE            3
Muscat, Oman              2
Al Rayyan, Qatar          2
Sharjah, UAE              2
Jeddah, Saudi Arabia      1
Kuwait City, Kuwait       1
Name: count, dtype: int64
